In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max as spark_max, min as spark_min, datediff, current_date
from pyspark.sql.types import DateType

In [2]:
# Tạo Spark session
spark = SparkSession.builder.appName("Big Data Project").getOrCreate()

24/12/15 14:49:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# <span style="color:red; background-color:yellow">Load data</span>


In [3]:
# Tải dữ liệu
df = spark.read.csv("customer.csv", header=True, inferSchema=True)
print("Number of datapoints:", df.count())
df.show(5)
df.printSchema()

Number of datapoints: 2240


24/12/15 14:49:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

# <span style="color:red; background-color:yellow">Data Cleaning</span>

In [4]:
df = df.dropna()
print("After removing NA values:", df.count())

After removing NA values: 2216


In [5]:
from pyspark.sql.functions import to_date

# Chuyển đổi sang DateType với định dạng chính xác
df = df.withColumn("Dt_Customer", to_date(col("Dt_Customer"), "dd-MM-yyyy"))

In [6]:
newest_date = df.agg(spark_max("Dt_Customer")).first()[0]
oldest_date = df.agg(spark_min("Dt_Customer")).first()[0]
print("The newest customer's enrolment date:", newest_date)
print("The oldest customer's enrolment date:", oldest_date)

The newest customer's enrolment date: 2014-06-29
The oldest customer's enrolment date: 2012-07-30


In [7]:
from pyspark.sql.functions import datediff, lit
# Tạo cột Customer_For, tính số ngày từ ngày mới nhất đến ngày của mỗi khách hàng
df = df.withColumn("Customer_For", datediff(lit(newest_date), col("Dt_Customer")))

In [8]:
from pyspark.sql.functions import when, expr

# Tạo cột Age (tuổi của khách hàng)
df = df.withColumn("Age", lit(2024) - col("Year_Birth"))

# Tính tổng chi tiêu cho các mục khác nhau
df = df.withColumn("Spent", 
                   col("MntWines") + col("MntFruits") + col("MntMeatProducts") +
                   col("MntFishProducts") + col("MntSweetProducts") + col("MntGoldProds"))

# Xác định tình trạng sống qua Marital_Status
df = df.withColumn("Living_With", 
                   when(col("Marital_Status").isin("Married", "Together"), "Partner")
                   .otherwise("Alone"))

# Tạo cột tổng số con trong hộ gia đình
df = df.withColumn("Children", col("Kidhome") + col("Teenhome"))

# Tạo cột tổng số thành viên trong hộ gia đình
df = df.withColumn("Family_Size", 
                   when(col("Living_With") == "Alone", 1)
                   .otherwise(2) + col("Children"))

# Tạo cột xác định khách hàng có phải là cha mẹ hay không
df = df.withColumn("Is_Parent", when(col("Children") > 0, 1).otherwise(0))

# Nhóm trình độ học vấn thành ba nhóm
df = df.withColumn("Education", 
                   when(col("Education").isin("Basic", "2n Cycle"), "Undergraduate")
                   .when(col("Education") == "Graduation", "Graduate")
                   .when(col("Education").isin("Master", "PhD"), "Postgraduate"))

# Đổi tên các cột để rõ ràng hơn
df = df.withColumnRenamed("MntWines", "Wines") \
       .withColumnRenamed("MntFruits", "Fruits") \
       .withColumnRenamed("MntMeatProducts", "Meat") \
       .withColumnRenamed("MntFishProducts", "Fish") \
       .withColumnRenamed("MntSweetProducts", "Sweets") \
       .withColumnRenamed("MntGoldProds", "Gold")

# Xóa các cột không cần thiết
columns_to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth"]
df = df.drop(*columns_to_drop)

df_cleaned = df
# Hiển thị kết quả
df.show(5)


+----+------------+------+-------+--------+-------+-----+------+----+----+------+----+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+--------+------------+---+-----+-----------+--------+-----------+---------+
|  ID|   Education|Income|Kidhome|Teenhome|Recency|Wines|Fruits|Meat|Fish|Sweets|Gold|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Response|Customer_For|Age|Spent|Living_With|Children|Family_Size|Is_Parent|
+----+------------+------+-------+--------+-------+-----+------+----+----+------+----+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+--------+------------+---+-----+-----------+--------+-----------+---------+
|5524|    Graduate| 58

# <span style="color:red; background-color:yellow">Data Exploration</span>

### Categorical Columns

In [9]:
# Đếm số lượng bản ghi và số lượng cột
print("Number of rows:", df.count())
print("Number of columns:", len(df.columns))

Number of rows: 2216
Number of columns: 31


In [10]:
from pyspark.sql.functions import col, count

categorical_cols = ["Education", "Living_With"]

for c in categorical_cols:
    print(f"Frequency table for {c}:")
    df.groupBy(c).count().orderBy("count", ascending=False).show()

Frequency table for Education:
+-------------+-----+
|    Education|count|
+-------------+-----+
|     Graduate| 1116|
| Postgraduate|  846|
|Undergraduate|  254|
+-------------+-----+

Frequency table for Living_With:
+-----------+-----+
|Living_With|count|
+-----------+-----+
|    Partner| 1430|
|      Alone|  786|
+-----------+-----+



### Numberic Columns

In [11]:
def printMedian(numeric_cols):
    for c in numeric_cols:
        median = df.approxQuantile(c, [0.5], 0.001)[0]
        print(f"Median of {c}: {median}")

In [12]:
numeric_cols = ['Income', 'Recency', 'Wines', 'Fruits', 'Meat', 'Fish']

df.select(numeric_cols).describe().show()
printMedian(numeric_cols)

+-------+------------------+-----------------+------------------+------------------+------------------+-----------------+
|summary|            Income|          Recency|             Wines|            Fruits|              Meat|             Fish|
+-------+------------------+-----------------+------------------+------------------+------------------+-----------------+
|  count|              2216|             2216|              2216|              2216|              2216|             2216|
|   mean| 52247.25135379061|49.01263537906137|305.09160649819495|26.356046931407942|166.99593862815885|37.63763537906137|
| stddev|25173.076660901414|28.94835165862499|  337.327920116672|39.793916518238674|224.28327275898198|54.75208241485154|
|    min|              1730|                0|                 0|                 0|                 0|                0|
|    max|            666666|               99|              1493|               199|              1725|              259|
+-------+---------------

In [13]:
numeric_cols = ['Sweets','Gold', 'Spent', 'Customer_For', 'Age', 'NumDealsPurchases']

df.select(numeric_cols).describe().show()
printMedian(numeric_cols)

+-------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|            Sweets|             Gold|            Spent|      Customer_For|               Age| NumDealsPurchases|
+-------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|              2216|             2216|             2216|              2216|              2216|              2216|
|   mean|27.028880866425993|43.96525270758123|607.0753610108303| 353.5212093862816| 55.17960288808664|2.3235559566787005|
| stddev| 41.07204566698224|51.81541437447363|602.9004763523358|202.43466744562002|11.985554097352384|1.9237156460114953|
|    min|                 0|                0|                5|                 0|                28|                 0|
|    max|               262|              321|             2525|               699|               131|                15|
+-------+---------------

In [14]:
numeric_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                'NumWebVisitsMonth', 'Children', 'Family_Size', 'Is_Parent']

df.select(numeric_cols).describe().show()
printMedian(numeric_cols)

+-------+------------------+-------------------+-----------------+-----------------+------------------+------------------+-------------------+
|summary|   NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|          Children|       Family_Size|          Is_Parent|
+-------+------------------+-------------------+-----------------+-----------------+------------------+------------------+-------------------+
|  count|              2216|               2216|             2216|             2216|              2216|              2216|               2216|
|   mean|  4.08528880866426|  2.671028880866426|5.800992779783393|5.319043321299639| 0.947202166064982| 2.592509025270758| 0.7143501805054152|
| stddev|2.7409510810147255| 2.9267336360080134|3.250784785670525|2.425358540606443|0.7490619124789906|0.9057218788428764|0.45182532445556944|
|    min|                 0|                  0|                0|                0|                 0|                 1|                  0|

In [15]:
numeric_cols = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                'AcceptedCmp2', 'Complain', 'Response']

df.select(numeric_cols).describe().show()
printMedian(numeric_cols)

+-------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|summary|       AcceptedCmp3|      AcceptedCmp4|       AcceptedCmp5|       AcceptedCmp1|        AcceptedCmp2|           Complain|           Response|
+-------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|  count|               2216|              2216|               2216|               2216|                2216|               2216|               2216|
|   mean|0.07355595667870037|0.0740072202166065|0.07310469314079422|0.06407942238267147|0.013537906137184115|0.00947653429602888|0.15027075812274368|
| stddev|0.26110580876067324|0.2618417283260366|0.26036702668143424| 0.2449496396591494| 0.11558832299800825| 0.0969070038487211|0.35741727012906527|
|    min|                  0|                 0|                  0|                  0|            

In [16]:
from pyspark.sql import Row

# Hàm tìm giá trị phổ biến nhất và hiếm nhất của một cột
def get_most_and_least_common(df, col_name):
    counts = df.groupBy(col_name).count().orderBy("count", ascending=False)
    most_common = counts.first()
    least_common = counts.orderBy("count", ascending=True).first()
    return most_common, least_common

# Danh sách các cột để phân tích
attributes = ['Education', 'Living_With', 'Income', 'Recency', 'Wines', 'Fruits', 'Meat', 'Fish', 'Sweets', 
                'Gold', 'Spent', 'Customer_For', 'Age', 'NumDealsPurchases',
                'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                'NumWebVisitsMonth', 'Children', 'Family_Size', 'Is_Parent',
                'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                'AcceptedCmp2', 'Complain', 'Response']

rows = []

# Tính giá trị phổ biến nhất và hiếm nhất cho từng cột
for attr in attributes:
    most_common, least_common = get_most_and_least_common(df, attr)
    rows.append(Row(statistic=f"most_common_{attr}", value=most_common[attr], count=most_common['count']))
    rows.append(Row(statistic=f"least_common_{attr}", value=least_common[attr], count=least_common['count']))

# Tạo DataFrame từ kết quả
stats_df = spark.createDataFrame(rows)

# Hiển thị kết quả
stats_df.show(100)

+--------------------+-------------+-----+
|           statistic|        value|count|
+--------------------+-------------+-----+
|most_common_Educa...|     Graduate| 1116|
|least_common_Educ...|Undergraduate|  254|
|most_common_Livin...|      Partner| 1430|
|least_common_Livi...|        Alone|  786|
|  most_common_Income|         7500|   12|
| least_common_Income|        82582|    1|
| most_common_Recency|           56|   37|
|least_common_Recency|           44|   11|
|   most_common_Wines|            2|   42|
|  least_common_Wines|          463|    1|
|  most_common_Fruits|            0|  395|
| least_common_Fruits|          101|    1|
|    most_common_Meat|            7|   53|
|   least_common_Meat|          471|    1|
|    most_common_Fish|            0|  379|
|   least_common_Fish|           81|    1|
|  most_common_Sweets|            0|  413|
| least_common_Sweets|          128|    1|
|    most_common_Gold|            1|   71|
|   least_common_Gold|          148|    1|
|   most_co

# <span style="color:red; background-color:yellow">Data PreProcessing</span>

In [17]:
from pyspark.sql.functions import col, when

# Giả sử numeric_cols là danh sách các cột số trong DataFrame df
# Đây là những cột mà bạn muốn xử lý ngoại lệ.
numeric_cols = ['Income', 'Recency', 'Wines', 'Fruits', 'Meat', 'Fish', 'Sweets', 
                'Gold', 'Spent', 'Customer_For', 'Age', 'NumDealsPurchases',
                'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                'NumWebVisitsMonth', 'Children', 'Family_Size']

# Xử lý ngoại lệ cho mỗi cột số
for c in numeric_cols:
    # Tính Q1 và Q3 sử dụng approxQuantile
    # Sai số 0.001 để đảm bảo độ chính xác tương đối cao
    q1, q3 = df.approxQuantile(c, [0.25, 0.75], 0.001)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Cắt giá trị ngoại lệ:
    # - Nếu giá trị < lower_bound => gán bằng lower_bound
    # - Nếu giá trị > upper_bound => gán bằng upper_bound
    # Giữ nguyên nếu nằm trong khoảng [lower_bound, upper_bound]
    df = df.withColumn(
        c,
        when(col(c) < lower_bound, lower_bound)
        .when(col(c) > upper_bound, upper_bound)
        .otherwise(col(c))
    )

### Z-score Normalization

In [18]:
from pyspark.sql.functions import mean as spark_mean, stddev as spark_stddev, col

numeric_cols = ["Income", "Kidhome", "Teenhome", "Recency", "Wines", "Fruits", "Meat", "Fish", "Sweets",
               "Gold", "Spent", "Customer_For", "Age", "NumDealsPurchases",
               "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases",
               "NumWebVisitsMonth", "Children", "Family_Size", "Is_Parent",
               "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "AcceptedCmp1",
                "AcceptedCmp2", "Complain", "Response"]

# Tính toán mean và stddev cho tất cả các cột số
agg_exprs = [spark_mean(c).alias(f"{c}_mean") for c in numeric_cols] + \
            [spark_stddev(c).alias(f"{c}_std") for c in numeric_cols]

stats_df = df.agg(*agg_exprs)

# Chuyển đổi stats_df sang dictionary để sử dụng trong việc chuẩn hóa
stats = stats_df.first().asDict()

# Tạo các cột chuẩn hóa Z-score: (x - mean) / std
for c in numeric_cols:
    mean_col = stats[f"{c}_mean"]
    std_col = stats[f"{c}_std"]
    df = df.withColumn(f"{c}_norm", (col(c) - mean_col) / std_col)

# Danh sách các cột chuẩn hóa
norm_cols = [f"{c}_norm" for c in numeric_cols]

# Loại bỏ các cột số gốc
df = df.drop(*numeric_cols)

In [19]:
df.show()

+----+-------------+-----------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+------------------------+----------------------+----------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
|  ID|    Education|Living_With|        Income_norm|       Kidhome_norm|      Teenhome_norm|       Recency_norm|          Wines_norm|         Fruits_norm|           Meat_norm|           Fish_norm|         Sweets_norm|           Gold_norm|          Spent_norm|   Customer_For_norm|            Age_norm|NumDealsPurchases_norm|NumWebPurchases_norm|NumCatalogPurchases_norm|NumStoreP

In [20]:
from pyspark.sql.functions import when

# Các cột phân loại
categorical_cols = ["Education", "Living_With"]

# Mã hóa one-hot cho cột 'Education'
df = df.withColumn("Education_Graduate", when(col("Education") == "Graduate", 1).otherwise(0))
df = df.withColumn("Education_Postgraduate", when(col("Education") == "Postgraduate", 1).otherwise(0))
df = df.withColumn("Education_Undergraduate", when(col("Education") == "Undergraduate", 1).otherwise(0))

# Mã hóa one-hot cho cột 'Living_With'
df = df.withColumn("Living_With_Alone", when(col("Living_With") == "Alone", 1).otherwise(0))
df = df.withColumn("Living_With_Partner", when(col("Living_With") == "Partner", 1).otherwise(0))

# Loại bỏ các cột phân loại gốc
df = df.drop(*categorical_cols)

df.show(5)

+----+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+----------------------+--------------------+------------------------+----------------------+----------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------+----------------------+-----------------------+-----------------+-------------------+
|  ID|        Income_norm|       Kidhome_norm|      Teenhome_norm|       Recency_norm|          Wines_norm|        Fruits_norm|           Meat_norm|          Fish_norm|         Sweets_norm|          Gold_norm|         Spent_norm|   Customer_For_norm|           Age_norm|NumDealsPurchases_norm|NumWebPurchas

In [21]:
# Các cột đã mã hóa
encoded_cols = ["Education_Graduate", "Education_Postgraduate", "Education_Undergraduate",
               "Living_With_Alone", "Living_With_Partner"]

# Danh sách tất cả các cột đặc trưng
feature_cols = norm_cols + encoded_cols

# Chuyển đổi dữ liệu thành RDD chứa (ID, vector)
rdd_points = df.select("ID", *feature_cols).rdd.map(lambda row: (row[0], [float(x) for x in row[1:]]))

In [22]:
import math

def euclidean_distance(p1, p2):
    return math.sqrt(sum((a - b)**2 for a, b in zip(p1, p2)))

def assign_to_cluster(point, centroids):
    distances = [euclidean_distance(point, c) for c in centroids]
    cluster_id = distances.index(min(distances))
    return cluster_id

def compute_sse(rdd_points, centroids):
    return rdd_points.map(lambda x: min([euclidean_distance(x[1], c)**2 for c in centroids])).sum()

# <span style="color:red; background-color:yellow">Data Mining</span>

In [24]:
# Định nghĩa phạm vi các giá trị k để thử
k_values = range(2, 11)  # Từ k=2 đến k=10

# Khởi tạo danh sách để lưu kết quả SSE cho mỗi k
sse_results = []

# Định nghĩa các tham số chung
max_iterations = 20
convergence_threshold = 1e-4

# Lặp qua từng giá trị k để thực hiện K-Means và tính SSE
for k in k_values:
    print(f"Running K-Means for k={k}")
    
    # Khởi tạo centroids ngẫu nhiên từ dữ liệu
    centroids = rdd_points.map(lambda x: x[1]).takeSample(False, k, seed=42)
    
    for i in range(max_iterations):
        # Gán cụm cho mỗi điểm
        clusters = rdd_points.map(lambda x: (assign_to_cluster(x[1], centroids), (x[1], 1)))
        
        # Tính centroid mới bằng cách lấy trung bình các điểm trong cụm
        new_centroids = clusters.reduceByKey(
            lambda a, b: ([x + y for x, y in zip(a[0], b[0])], a[1] + b[1])
        ).mapValues(
            lambda x: [val / x[1] for val in x[0]]
        )
        
        # Chuyển centroids thành list để dễ dàng so sánh
        new_centroids_list = new_centroids.collect()
        new_centroids_dict = dict(new_centroids_list)
        
        # Kiểm tra hội tụ: tính khoảng cách giữa centroids cũ và mới
        max_distance = 0
        for i, centroid in enumerate(centroids):
            if i in new_centroids_dict:
                distance = euclidean_distance(centroid, new_centroids_dict[i])
                max_distance = max(max_distance, distance)
        
        # Kiểm tra hội tụ
        if max_distance < convergence_threshold:
            print(f"  Converged at iteration: {i+1}")
            centroids = [new_centroids_dict[key] for key in sorted(new_centroids_dict.keys())]
            break
        
        centroids = [new_centroids_dict[key] for key in sorted(new_centroids_dict.keys())]

    # Tính SSE cho k hiện tại (giữ RDD phân tán)
    sse = rdd_points.map(lambda x: min([euclidean_distance(x[1], c)**2 for c in centroids])).sum()
    print(f"  SSE for k={k}: {sse}")
    
    # Lưu kết quả
    sse_results.append((k, sse))

# Tạo DataFrame từ kết quả SSE
sse_df = spark.createDataFrame(sse_results, ["k", "SSE"])

# Hiển thị kết quả SSE cho từng k
sse_df.orderBy("k").show()


Running K-Means for k=2
  Converged at iteration: 2
  SSE for k=2: 48215.17278440477
Running K-Means for k=3
  Converged at iteration: 3
  SSE for k=3: 43188.20242947265
Running K-Means for k=4
  SSE for k=4: 40848.80333983346
Running K-Means for k=5
  SSE for k=5: 38559.207199458324
Running K-Means for k=6
  Converged at iteration: 6
  SSE for k=6: 37493.3884607082
Running K-Means for k=7
  Converged at iteration: 7
  SSE for k=7: 36305.355804365776
Running K-Means for k=8
  SSE for k=8: 35394.322807038356
Running K-Means for k=9
  SSE for k=9: 33579.82712514724
Running K-Means for k=10
  SSE for k=10: 32721.91198359116
+---+------------------+
|  k|               SSE|
+---+------------------+
|  2| 48215.17278440477|
|  3| 43188.20242947265|
|  4| 40848.80333983346|
|  5|38559.207199458324|
|  6|  37493.3884607082|
|  7|36305.355804365776|
|  8|35394.322807038356|
|  9| 33579.82712514724|
| 10| 32721.91198359116|
+---+------------------+



In [26]:
# Chọn số cụm k tối ưu (ví dụ: k=3)
optimal_k = 3

print(f"Running K-Means for optimal k={optimal_k}")

# Khởi tạo centroids ngẫu nhiên từ dữ liệu
centroids = rdd_points.map(lambda x: x[1]).takeSample(False, k, seed=42)
    
for i in range(max_iterations):
    # Gán cụm cho mỗi điểm
    clusters = rdd_points.map(lambda x: (assign_to_cluster(x[1], centroids), (x[1], 1)))
    
    # Tính centroid mới bằng cách lấy trung bình các điểm trong cụm
    new_centroids = clusters.reduceByKey(
        lambda a, b: ([x + y for x, y in zip(a[0], b[0])], a[1] + b[1])
    ).mapValues(
        lambda x: [val / x[1] for val in x[0]]
    )
    
    # Chuyển centroids thành list để dễ dàng so sánh
    new_centroids_list = new_centroids.collect()
    new_centroids_dict = dict(new_centroids_list)
    
    # Kiểm tra hội tụ: tính khoảng cách giữa centroids cũ và mới
    max_distance = 0
    for i, centroid in enumerate(centroids):
        if i in new_centroids_dict:
            distance = euclidean_distance(centroid, new_centroids_dict[i])
            max_distance = max(max_distance, distance)
    
    # Kiểm tra hội tụ
    if max_distance < convergence_threshold:
        print(f"  Converged at iteration: {i+1}")
        centroids = [new_centroids_dict[key] for key in sorted(new_centroids_dict.keys())]
        break
    
    centroids = [new_centroids_dict[key] for key in sorted(new_centroids_dict.keys())]

# In kết quả centroids cuối cùng
print("Final centroids:", centroids)

Running K-Means for optimal k=3
Final centroids: [[0.41883350565211014, -0.5058225730627982, 0.732917910864925, 0.09327406997907427, 1.0324909239860547, -0.3458440714379548, -0.260426386536874, -0.42969863484890286, -0.48495030134187717, -0.0495612299625837, 0.3696286214404407, 0.0819810761650904, 0.27161285664171914, 0.6002601896582098, 0.7141955568844719, 0.130337094891355, 0.5651881664562606, 0.3884274249765731, 0.17733192142342766, 0.1371769883644164, 0.3967613833668789, -0.2817093845128546, 3.5364599283059257, 0.08695412778965374, 0.12927252521152885, 0.6191670143252478, -0.09778998338265053, 0.17485220930006387, 0.43617021276595747, 0.5212765957446809, 0.0425531914893617, 0.3617021276595745, 0.6382978723404256], [-1.0170873464769947, 1.0664389514665131, -0.9023852442080509, 0.02890136638712812, -0.8232686548262095, -0.5898670913558877, -0.7022597506727677, -0.591884935217293, -0.5885023913106824, -0.6390802840906288, -0.8541809668624512, -0.10985723451512049, -0.8096098426106116,

In [ ]:

# Gán cụm cho mỗi điểm với k tối ưu
cluster_assignments = rdd_points.map(lambda x: (x[0], assign_to_cluster(x[1], centroids)))
clusters_df = cluster_assignments.toDF(["ID", "cluster_id"])

# Đọc lại DataFrame gốc (giả sử bạn đã có DataFrame gốc trước khi chuẩn hóa và mã hóa)
# Nếu bạn đã giữ lại DataFrame gốc trong biến df_original, sử dụng biến đó
# Nếu không, đọc lại từ file CSV hoặc nơi bạn lưu trữ DataFrame gốc
# Giả sử DataFrame gốc là df_original
# df_original = spark.read.csv("original_data.csv", header=True, inferSchema=True)

# Gán nhãn cụm vào DataFrame gốc
final_df = df_cleaned.join(clusters_df, on="ID", how="inner")

# Kiểm tra DataFrame cuối cùng
final_df.select("ID", "cluster_id").show(5)

# Xuất DataFrame cuối cùng ra file CSV
final_df.write.csv("output_with_clusters.csv", header=True)